In [1]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
import pandas as pd
from scipy.special import softmax
import csv
import urllib.request

/Users/daniel/miniforge3/envs/GPT/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Load CSV file and create DataFrame of Discord chat


In [7]:
data = pd.read_csv("DiscordData.csv")
data.head()

,AuthorID,Author,Date,Content,Attachments,Reactions
0,370963041141129226,Daiigr#7964,02-Jul-20 08:53 PM,who wants to play something?,NaN,NaN
1,607796009707372554,Emo Knightmare#5231,02-Jul-20 10:17 PM,Sorry fam no gaming after 8 PM courtesy of my ...,NaN,NaN
2,370963041141129226,Daiigr#7964,03-Jul-20 12:26 AM,Oof,NaN,NaN
3,537937247891357697,The Dark Soul#7970,03-Jul-20 12:31 AM,big sad,NaN,NaN
4,503221991717863424,LET ME IN!!!#0749,03-Jul-20 01:14 AM,If anyone here plays DBD and wants to have som...,NaN,NaN


### removing Bot messages

In [13]:
cleaned_data = data

cleaned_data = cleaned_data.drop(cleaned_data[cleaned_data['Author'] == 'Rythm#3722'].index)

cleaned_data = cleaned_data.drop(cleaned_data[cleaned_data['Author'] == 'Loli Police#5803'].index)

cleaned_data = cleaned_data.drop(cleaned_data[cleaned_data['Author'] == 'Cool Frog Facts #frogs#0000'].index)

cleaned_data = cleaned_data.drop(cleaned_data[cleaned_data['Author'] == 'Minecaft Bot#1886'].index)

cleaned_data = cleaned_data.drop(cleaned_data[cleaned_data['Author'] == 'Dank Memer#5192' ].index)

cleaned_data = cleaned_data.drop(cleaned_data[cleaned_data['Author'] == 'Simple Poll#9879' ].index)

cleaned_data = cleaned_data.drop(cleaned_data[cleaned_data['Author'] == 'UnoBot#6134' ].index)

cleaned_data = cleaned_data.drop(cleaned_data[cleaned_data['Author'] == 'MEE6#4876' ].index)

cleaned_data = cleaned_data.drop(cleaned_data[cleaned_data['Author'] == 'Daiigr AI#4911' ].index)


print(cleaned_data['Author'].unique())

print(len(data))
print(len(cleaned_data))

['Daiigr#7964' 'Emo Knightmare#5231' 'The Dark Soul#7970'
 'LET ME IN!!!#0749' 'PETTAPLAY#7395' 'kima#1596' 'haani_05#5594'
 'Deleted User#0000' 'existential.duck#5117' 'Eeugh#0033'
 'Master chief#7460' 'KyroGaming#1987' 'soumar#3751' 'Iann#4893'
 'Haruka Sakura#2591' 'schrimpf#2940' 'Dwonks#8334' 'hagha#2602'
 'carlitos#1736' 'rY4n#8975' 'seafox2205#6878' 'Father#2152'
 'XxEXO_TROOPERxX#8603' "I'll do my best#9945" 'Peachymango11#3226'
 'lana#8237' 'sofia719462#2756' '(G)raveyard #dear-god#0000' 'Son#9990'
 'Lord Lazinga#7942' 'neurotell#9825' 'i have name#0161' 'Caitlin#6324']
187600
184938


# Preprocess text (username and link placeholders)


In [2]:
def preprocess(text):
    new_text = []
 
 
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

# Tasks:
emoji, emotion, hate, irony, offensive, sentiment
stance/abortion, stance/atheism, stance/climate, stance/feminist, stance/hillary


In [3]:
task='sentiment'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"

tokenizer = AutoTokenizer.from_pretrained(MODEL)

In [4]:
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
model.save_pretrained(MODEL)

# download label mapping

In [38]:
labels=[]
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
labels = [row[1] for row in csvreader if len(row) > 1]

KeyboardInterrupt: 

In [5]:
def listToString(instr):
        emptystr=""
        for ele in instr: 
            emptystr += ele + " "
        return emptystr

# run though each user though the model

## Exploring our Data 

lets look how many messages each user has sent in this discord server. later on we will use this data to see if there is any corrolation between average sentiment and user messages

In [ ]:
from IPython.display import clear_output
NumberOfMessages = []
avgUserScore = []
    
for Author in cleaned_data['Author'].unique():
    
    usermessage = cleaned_data[cleaned_data['Author'] == Author ]
    try:
     usermessage =  usermessage.sample(n=50)
    except:
        usermessage = usermessage
    #convert from dataframe to array of only the content of the message   
    UserContent = usermessage['Content'].copy()
    UserContent = UserContent.values
    #make sure all objects are strings and not any other type
    StrUserContent = []
    for line in UserContent:
        StrUserContent.append(str(line))

    Scores_arr = []
    for currentline in StrUserContent:
        text = currentline
        text = preprocess(text)
        encoded_input = tokenizer(text, return_tensors='pt')
        output = model(**encoded_input)
        scores = output[0][0].detach().numpy()
        Scores_arr.append(softmax(scores))
        
        print(Author + ': current Sample index' + str(len(Scores_arr)))
    
    totalScore = 0
    avgScore = 0
    #calcuate the average between the list of scores calulated for that user
    for i in Scores_arr:
        totalScore += i
    
    avgScore = totalScore/len(Scores_arr)
    avgUserScore.append([Author,avgScore])
#print array of avg user scores for all users
clear_output(wait=True)
print(avgUserScore)

# show the Dataframe

In [2]:
from time import sleep
from tqdm.notebook import tqdm
for i in tqdm(range(10)):
    sleep(3)

ImportError: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html

In [39]:
import plotly.express as px
fig = px.bar(NOMdf, x='Name', y='NumberOfMessages',template = "plotly_dark", labels={'NumberOfMessages':'Number of Total Messages','Name':'Discord UserName'})
fig.show()

,Name,NumberOfMessages
3,LET ME IN!!!#0749,24528
19,rY4n#8975,6
12,soumar#3751,79938


In [29]:
StrContent = []
for line in Content:
    StrContent.append(str(line))

Scores_arr = []
for i in StrContent:
    text = i
    text = preprocess(text)
    encoded_input = tokenizer(text, return_tensors='pt')
    output = model(**encoded_input)
    scores = output[0][0].detach().numpy()
    Scores_arr.append(softmax(scores))
    print('Current Score array length : ' +str(len(Scores_arr)))
    if(len(Scores_arr) > 3000):
        break


Current Score array length : 1
Current Score array length : 2
Current Score array length : 3
Current Score array length : 4
Current Score array length : 5
Current Score array length : 6
Current Score array length : 7
Current Score array length : 8
Current Score array length : 9
Current Score array length : 10
Current Score array length : 11
Current Score array length : 12
Current Score array length : 13
Current Score array length : 14
Current Score array length : 15
Current Score array length : 16
Current Score array length : 17
Current Score array length : 18
Current Score array length : 19
Current Score array length : 20
Current Score array length : 21
Current Score array length : 22
Current Score array length : 23
Current Score array length : 24
Current Score array length : 25
Current Score array length : 26
Current Score array length : 27
Current Score array length : 28
Current Score array length : 29
Current Score array length : 30
Current Score array length : 31
Current Score arr

In [30]:
for i in Scores_arr:
    i[0] *= -1 

In [31]:

df = pd.DataFrame(Scores_arr, columns = ['Negative', 'Neutral', 'Positive'])
df.head()

,Negative,Neutral,Positive
0,-0.033745,0.774871,0.191383
1,-0.669473,0.309760,0.020767
2,-0.685864,0.279974,0.034162
3,-0.818575,0.156068,0.025357
4,-0.765772,0.220668,0.013560


In [32]:
import plotly.express as px


fig = px.scatter(df, y=['Negative','Neutral','Positive'],template = "plotly_dark",trendline="lowess")
fig.show() 

In [33]:
fig = px.scatter(df, trendline="lowess",template = "plotly_dark",
                title="sentiment analysis of discord server data")
fig.data = [t for t in fig.data if t.mode == "lines"]
fig.update_traces(showlegend=True) #trendlines have showlegend=False by default
fig.show()